# json2mmaction2
version: 1

info:
- Create standard MOLA JSON

author: nuno costa

In [21]:
import json
import os
import argparse
import time
import copy
import glob
import shutil
from pathlib import Path
import numpy as np
from tqdm import tqdm

In [3]:
def make_folders(path='../out/'):
    # Create folders
    if os.path.exists(path):
        shutil.rmtree(path)  # delete output folder
    os.makedirs(path)  # make new output folder
    return

In [4]:
def extract_file(src,dst,copy=True):
    extracted=True
    try:
        if copy: 
            if not os.path.exists(os.path.dirname(dst)): 
                os.makedirs(os.path.dirname(dst)) #make sure dir exists
            shutil.copyfile(src, dst)  # raises if missing files
        else: #if not copy only extracting filelist from json
            if not os.path.exists(src): raise
    except:
        print("\n>> missing : {}".format(src))
        extracted=False
    return extracted
#extract_file("/home/administrator/server_data_ext/Recordings/EASYRIDE/P19/INCAR/20210422/Session1/C1_P7_P8_1/gt/rgb/1.png","/home/administrator/Desktop/INCAR_20210427_Session_1_C10_P13_P14_1_rgb/img_1.png")

In [6]:
def rename_filepaths_from_dir(path):
    #because of possible missing files or miss placement of total frames
    for subdir, dirs, files in os.walk(path):
        print(subdir)
        #print(dirs)
        print(len(files))
        files.sort()
        for i,file in enumerate(files):
            if file.endswith(('.png')):
                newfile="img_"+str(i+1).zfill(5)+".png" 
                if file==newfile: continue
                src = os.path.join(subdir,file)
                dst = os.path.join(subdir,"rename_"+newfile) #rename_ is necessary for not substituing previous file
                try:
                    os.rename(src,dst)
                    print("\n>> rename success: from {} to {}".format(file,newfile))
                except:
                    print("\n>> rename failed: {}".format(src))
                    continue
    #remove rename_
    for subdir, dirs, files in os.walk(path):
        for i,file in enumerate(files):
            if file.endswith(('.png')) and file.startswith(('rename_')) :
                newfile = file.replace('rename_','')
                src = os.path.join(subdir,file)
                dst = os.path.join(subdir,newfile)
                try:
                    os.rename(src,dst)
                except:
                    print("\n>> rename failed: {}".format(src))
                    continue               
    return
path='/home/user/Data/Algorithms/mmaction2/data/INCAR/rawframes'
replace_filepaths_from_dir(path)

/home/user/Data/Algorithms/mmaction2/data/INCAR/rawframes
0
/home/user/Data/Algorithms/mmaction2/data/INCAR/rawframes/NONVIOLENT
0
/home/user/Data/Algorithms/mmaction2/data/INCAR/rawframes/NONVIOLENT/INCAR_20210422_Session1_C10_P7_P8_1_rgb_1
441
/home/user/Data/Algorithms/mmaction2/data/INCAR/rawframes/NONVIOLENT/INCAR_20210422_Session1_C10_P7_P8_2_rgb_3
413
/home/user/Data/Algorithms/mmaction2/data/INCAR/rawframes/NONVIOLENT/INCAR_20210422_Session1_C10_P8_P7_1_rgb_5
403
/home/user/Data/Algorithms/mmaction2/data/INCAR/rawframes/NONVIOLENT/INCAR_20210422_Session1_C10_P8_P7_2_rgb_7
390
/home/user/Data/Algorithms/mmaction2/data/INCAR/rawframes/NONVIOLENT/INCAR_20210422_Session1_C11_P7_P8_1_rgb_9
506
/home/user/Data/Algorithms/mmaction2/data/INCAR/rawframes/NONVIOLENT/INCAR_20210422_Session1_C11_P7_P8_2_rgb_11
465
/home/user/Data/Algorithms/mmaction2/data/INCAR/rawframes/NONVIOLENT/INCAR_20210422_Session1_C11_P8_P7_1_rgb_13
504
/home/user/Data/Algorithms/mmaction2/data/INCAR/rawframes/NONV

# SINGLE CASE STUDY

In [5]:
datasets_root_dir='D:/external_datasets/MOLA/'
json_dir='D:/external_datasets/MOLA/INCAR/'
outdir='D:/external_datasets/MOLA/INCAR_MMA/' #don't forget the last
make_folders(path=outdir)  # output directory
dataset="mola"
json_file = json_dir+'mola.json'
img_number=None #STOP CONDITION : None copies all
copy_images=True #if false only filelist is extracted from json
copy_videos=False
level=2 #level: 1- one level; 2-two-level

### PREPARE DATASET

In [11]:
def write_from_annotation(json_file, data, images, videos, categories, videos_ann, 
                          copy_images, copy_videos, outdir_img, outdir_video, 
                          img_number=None, level=2, min_frames=30):
    # WRITE FILES (COPY & GENERATE FILELIST)
    # image lists
    img_l = []
    saved_img_l= []
    imglist = []
    img_counter = 0 # image counter
    # video lists
    video_l = []
    saved_video_l= []
    videolist = []
    # write files 
    method="for" #TODO: parfor method
    start=time.time()
    if method=="for":
        #WRITE IMAGES
        for x in tqdm(data['annotations'], desc='Annotations %s' % json_file):  
            # extract label and category
            catid = '%g' % x['category_id']
            label = catid
            category = categories[label]['name']
            # extract image info from x['image_id']
            image_id='%g' % x['image_id']
            if image_id in img_l: continue # continue to next loop if repeated image_id 
            img_l.append(image_id)
            img = images[image_id]
            h, w, imgf = img['height'], img['width'], img['file_name']
            _, img_ext = os.path.splitext(imgf)
            img_fn = Path(imgf).stem
            img_frame_original = img['frame_index'] - 1  # zero-padding because frame starts in one
            # extract video info from img['video_id']
            video_id = '%g' % img["video_id"]
            video = videos[video_id]
            videof= video["name"]
            video_fn = Path(videof).stem
            keys_l=[k for k in list(videos_ann.keys()) if k.find('_'+video_id+'_'+catid)>-1]
            video_key=[k for k in keys_l if img_frame>=videos_ann[k]['frame_start'] and img_frame<=videos_ann[k]['frame_end']]
            if not video_key: continue #IMG FRAME DOES NOT BELONG TO THIS ANNOTATION
            if len(video_key)>1: raise AssertionError("Current rule - An image can only be in one annotation label!")
            video_key= video_key[0] #Only one 
            video_ann= videos_ann[video_key]
            video_start_frame=video_ann["frame_start"]-1 # zero-padding because frame starts in one
            video_end_frame=video_ann["frame_end"]-1 
            # extract total label frames
            total_frames = '%g' % video_ann['label_frames']
            if video_ann['label_frames']<30: continue
            # extract bounding box format is [top left x, top left y, width, height] | [x,y,w,h]
            box = np.array(x['bbox'], dtype=np.float64) 
            box[:2] += box[2:] / 2  # xy top-left corner to center
            box[[0, 2]] /= w  # normalize x & w
            box[[1, 3]] /= h  # normalize y & h
            if (box[2] > 0.) and (box[3] > 0.):  # if w > 0 and h > 0
                # New fnames
                img_frame=str(int(img_frame_original)-int(video_start_frame))#padding
                img_new_fn = "img_"+img_frame.zfill(5) #img_framenumber (imgid with zeros 00001: image_frame.zfill(5) ) #WARNING: Problem is I don't no the maximum of images at this point
                video_new_fn = video_fn+'_'+'%g' % video_ann['id'] #the video annotation id is obligatory
                # write images - 1st because if copy_images fails the rest should not be done
                src = os.path.join(datasets_root_dir, imgf)
                dst = os.path.join(outdir_img, video_new_fn, img_new_fn + img_ext)
                if level==2: dst = os.path.join(outdir_img, category, video_new_fn, img_new_fn + img_ext)
                ext=extract_file(src,dst,copy=copy_images)
                if not ext: continue #if image missing from dataset when extracting images dont write nothing more
                # img list: 
                imgline = f'{video_new_fn}/{img_new_fn}\n' # f'{video_new_fn}/{img_new_fn} {total_frames} {label}\n'
                if level==2: imgline = f'{category}/{video_new_fn}/{img_new_fn}\n' # f'{category}/{video_new_fn}/{img_new_fn} {total_frames} {label}\n'
                imglist.append(imgline)
                img_counter += 1
                # rawframe annotation file list: json to txt [ frame_directory total_frames label  ]
                vidline = f'{video_new_fn} {total_frames} {label}\n'
                if level==2: vidline = f'{category}/{video_new_fn} {total_frames} {label}\n' 
                videolist.append(vidline)
            # STOP conditions
            if img_number and img_counter >= img_number: 
                print("STOP CONDITION")
                break
        #remove duplicate paths
        imglist=list(dict.fromkeys(imglist)) 
        videolist=list(dict.fromkeys(videolist)) 
    stop = time.time()
    elapsed=stop-start
    print("time elapsed:", elapsed)
    return imglist, videolist, saved_img_l, saved_video_l

In [7]:
def mola2mmaction2(datasets_root_dir=None, json_file='mola.json', outdir='out/', copy_images=True, copy_videos=False, img_number=None, level=2):
    # MAKE ROOT DIRS
    videodir_path = 'videos_%s/' % Path(json_file).stem  # folder name (train, val, test) remove other info
    imgdir_path = 'rawframes_%s/' % Path(json_file).stem  # folder name (train, val, test) remove other info
    outdir_video = os.path.join (outdir, videodir_path)
    outdir_img = os.path.join (outdir, imgdir_path)
    if copy_videos: make_folders(path=outdir_video)
    if copy_images: make_folders(path=outdir_img)
    # PARSE JSON ANNOTATIONS
    data=None
    with open(json_file) as f:
        data = json.load(f)
    if not data: raise
    # create image dict {id: image}
    images = {'%g' % x['id']: x for x in data['images']}
    # create video dict {id: video}
    videos = {'%g' % x['id']: x for x in data['videos']}
    # create category dict {id: category}
    categories = {'%g' % x['id']: x for x in data['categories']}
    # create video ann dict {video_id_category_id: video_annotations}
    videos_ann = {'%g' % x['id'] +'_'+'%g' % x['video_id'] +'_'+'%g' % x['category_id']: x for x in data['video_annotations']}
    # WRITE FILES (COPY & GENERATE FILELIST)
    method="from_annotation"
    if method=="from_annotation": 
        imglist, videolist, saved_img_l, saved_video_l=write_from_annotation(json_file, data, images, videos, categories, videos_ann, copy_images, copy_videos, outdir_img, outdir_video, img_number=img_number, level=level)
    
    
    return imglist, videolist, saved_img_l, saved_video_l
imglist, videolist, saved_img_l, saved_video_l = mola2mmaction2(datasets_root_dir=datasets_root_dir, json_file=json_file, outdir=outdir, copy_images=copy_images, copy_videos=copy_videos, img_number=img_number, level=level)

FileNotFoundError: [Errno 2] No such file or directory: 'D:/external_datasets/MOLA/INCAR/mola.json'

### GENERATE FILELIST

In [7]:
# GENERATE FILELISTS
#save imglist : mola_{train,val}_rawframes.txt
dataset_type='rawframes'
filename=f'{dataset}_total{Path(json_file).stem}_{dataset_type}.txt'
with open(outdir + filename, 'w') as f:
    f.writelines(videolist)

In [23]:
def recheck_rawframes_filelist(rdir, totalfilelistname):
    with open(os.path.join(rdir, totalfilelistname)) as file: totalfilelist=file.readlines()
    #print(len(totalfilelist))
    original=copy.copy(totalfilelist)
    framesdir=os.path.join(rdir,'rawframes')
    for subdir, dirs, files in os.walk(framesdir):
        if files and files[0].endswith(('.png')):
            dirname=os.path.split(subdir)[-1]
            #print(dirname)
            #print(len(files))
            indices = [i for i, s in enumerate(totalfilelist) if dirname in s]
            #print(indices)
            for idx in indices:
                line=totalfilelist[idx].split(' ')
                line[-2]= str(len(files)) #update total_frames: f'{category}/{video_new_fn} {total_frames} {label}\n' 
                totalfilelist[idx]= ' '.join(line)
    newfilename=os.path.join(rdir, "recheck_"+totalfilelistname)
    with open(newfilename, 'w') as f:
        f.writelines(totalfilelist)  
    comparision=[original[i]==totalfilelist[i] for i,n in enumerate(original)]
    return comparision


rdir='/home/user/Data/Algorithms/mmaction2/data/INCAR'
totalfilelistname='INCAR_total_rawframes.txt'
comparision=recheck_rawframes_filelist(rdir, totalfilelistname) 
display(sum([not c for c in comparision]))

4

In [16]:
sum(not [False, True])

TypeError: 'bool' object is not iterable

### SPLIT FILELIST

In [14]:
def split_filelist(path):
    filelist=[]
    with open(path) as f:
        filelist=f.readlines()
    split=['train','val','test']
    for s in split:
        ps=[]
        if s=="train": ps=['P'+str(i+1)+'_' for i in range(8)]
        if s=="val": ps=['P'+str(i+1)+'_' for i in range(8,12)]
        if s=="test": ps=['P'+str(i+1)+'_' for i in range(12,16)]
        filelist_split=[l for l in filelist for p in ps if l.find(p)>-1]
        filelist_split=list(dict.fromkeys(filelist_split))
        fn_split=Path(path).stem.split('_')
        fn_split[-2]=s
        fn_split='_'.join(fn_split)
        
        with open(os.path.join(os.path.dirname(path),fn_split+'.txt'), 'w') as f:
            f.writelines(filelist_split)
    return
split_filelist("/home/administrator/Z/Algorithms/mmaction2/data/INCAR/INCAR_total_rawframes.txt")

### CONFIG MMACTION2
1. Option (RECOMMENDED): create a customconfig.py file 
2. Option: modify other config and train from script (see folder MMACTION2/TESTS/mmaction2_tutorial)

In [37]:
# 2. Option
from mmcv import Config
cfg = Config.fromfile('./configs/recognition/tsn/tsn_r50_video_1x1x8_100e_kinetics400_rgb.py')

Given a config that trains a TSN model on kinetics400-full dataset, we need to modify some values to use it for training TSN on mola dataset.


In [ ]:
root= outdir #default: "data/"
dataset="mola"
dataset_type = 'RawframeDataset'

# Modify dataset type and path
cfg.dataset_type = dataset_type
cfg.data_root =  root+dataset+imgdir_path
cfg.data_root_val = root+dataset+imgdir_path
cfg.ann_file_train =  root+dataset+'/'+dataset+'_train_rawframes.txt'
cfg.ann_file_val = root+dataset+'/'+dataset+'_val_rawframes.txt'
cfg.ann_file_test = root+dataset+'/'+dataset+'_val_rawframes.txt'

cfg.data.test.type = dataset_type
cfg.data.test.ann_file = root+dataset+'/'+dataset+'_val_rawframes.txt'
cfg.data.test.data_prefix = cfg.data_root_val #'data/kinetics400_tiny/val/'

cfg.data.train.type = dataset_type
cfg.data.train.ann_file = root+dataset+'/'+dataset+'_train_rawframes.txt'
cfg.data.train.data_prefix = cfg.data_root #'data/kinetics400_tiny/train/'

cfg.data.val.type = dataset_type
cfg.data.val.ann_file = root+dataset+'/'+dataset+'_val_rawframes.txt'
cfg.data.val.data_prefix = cfg.data_root_val #'data/kinetics400_tiny/val/'

# The flag is used to determine whether it is omnisource training
cfg.setdefault('omnisource', False)
# Modify num classes of the model in cls_head
cfg.model.cls_head.num_classes = 2
# We can use the pre-trained TSN model
cfg.load_from = './checkpoints/tsn_r50_1x1x3_100e_kinetics400_rgb_20200614-e508be42.pth'

# Set up working dir to save files and logs.
cfg.work_dir = './tutorial_exps'

# The original learning rate (LR) is set for 8-GPU training.
# We divide it by 8 since we only use one GPU.
cfg.data.videos_per_gpu = cfg.data.videos_per_gpu // 16
cfg.optimizer.lr = cfg.optimizer.lr / 8 / 16
cfg.total_epochs = 30

# We can set the checkpoint saving interval to reduce the storage cost
cfg.checkpoint_config.interval = 10
# We can set the log print interval to reduce the the times of printing log
cfg.log_config.interval = 5

# Set seed thus the results are more reproducible
cfg.seed = 0
set_random_seed(0, deterministic=False)
cfg.gpu_ids = range(1)


# We can initialize the logger for training and have a look
# at the final config used for training
print(f'Config:\n{cfg.pretty_text}')

### Train a new recognizer

Finally, lets initialize the dataset and recognizer, then train a new recognizer!

In [ ]:
import os.path as osp

from mmaction.datasets import build_dataset
from mmaction.models import build_model
from mmaction.apis import train_model

import mmcv

# Build the dataset
datasets = [build_dataset(cfg.data.train)]

# Build the recognizer
model = build_model(cfg.model, train_cfg=cfg.get('train_cfg'), test_cfg=cfg.get('test_cfg'))

# Create work_dir
mmcv.mkdir_or_exist(osp.abspath(cfg.work_dir))
train_model(model, datasets, cfg, distributed=False, validate=True)

## Test the trained recognizer

After finetuning the recognizer, let's check the prediction results!

In [ ]:
from mmaction.apis import single_gpu_test
from mmaction.datasets import build_dataloader
from mmcv.parallel import MMDataParallel

# Build a test dataloader
dataset = build_dataset(cfg.data.test, dict(test_mode=True))
data_loader = build_dataloader(
        dataset,
        videos_per_gpu=1,
        workers_per_gpu=cfg.data.workers_per_gpu,
        dist=False,
        shuffle=False)
model = MMDataParallel(model, device_ids=[0])
outputs = single_gpu_test(model, data_loader)

eval_config = cfg.evaluation
eval_config.pop('interval')
eval_res = dataset.evaluate(outputs, **eval_config)
for name, val in eval_res.items():
    print(f'{name}: {val:.04f}')

# MULTI CASE STUDY (For loop script for each json - train, val , test)

In [12]:
def convert_mola_json(dataset="mola", datasets_root_dir=None, json_dir='../mola/annotations/', outdir='out/', copy_images=True, copy_videos=False, img_number=None, level=2):
    # Convert motionLab JSON file into  labels --------------------------------
    make_folders(path=outdir)  # output directory
    jsons = glob.glob(json_dir + '*.json')
    # Import json
    for json_file in sorted(jsons):
        imglist, videolist, saved_img_l, saved_video_l = mola2mmaction2(datasets_root_dir=datasets_root_dir, 
                                                                        json_file=json_file, 
                                                                        outdir=outdir, 
                                                                        copy_images=copy_images, 
                                                                        copy_videos=copy_videos, 
                                                                        img_number=img_number,
                                                                        level=level
                                                                       )
        # GENERATE FILELISTS
        #save videolist : mola_{train,val}_rawframes.txt
        dataset_type='rawframes'
        filename=f'{dataset}_{Path(json_file).stem}_{dataset_type}.txt'
        # Recheck total frame numbers of list
        with open(outdir + filename, 'w') as f:
            f.writelines(videolist)
            
    # Recheck and rename if necessary
    rename_filepaths_from_dir(outdir)
    
    
    return


In [13]:
#WARNING use / in dir path if forlder
datasets_root_dir='/home/administrator/server_data_ext/Recordings/EASYRIDE/P19/'#'D:/external_datasets/MOLA/'
json_dir='/home/administrator/server_data_ext/Recordings/EASYRIDE/P19/INCAR/'#'D:/external_datasets/MOLA/INCAR/'
outdir='/home/administrator/Z/Algorithms/mmaction2/data/INCAR/'#'D:/external_datasets/MOLA/INCAR_MMA_FORMAT/'
img_number=None
copy_images=True
copy_videos=False
level=2
convert_mola_json(datasets_root_dir=datasets_root_dir, json_dir=json_dir, outdir=outdir, 
                  img_number=img_number, copy_images=copy_images, copy_videos=copy_videos, level=level)

Annotations /home/administrator/server_data_ext/Recordings/EASYRIDE/P19/INCAR/mola.json: 100%|██████████| 443433/443433 [9:14:50<00:00, 13.32it/s]   


time elapsed: 33290.56068563461


In [40]:
jsons = glob.glob(json_dir + '*.json')
json_file=jsons[0]
print(json_file)
with open(json_file) as f:
    data = json.load(f)
print(data['videos'][0])
# create video dict {id: video}
videos = {'%g' % x['id']: x for x in data['videos']}
print(videos['1'])
js=json.load(open(json_file))
print(js['videos'][0])

/home/administrator/server_data_ext/Recordings/EASYRIDE/P19/INCAR/mola.json
{'name': 'INCAR/20210422/Session1/C10_P7_P8_1/gt/INCAR_20210422_Session1_C10_P7_P8_1_rgb.mp4', 'id': 1, 'width': 2048, 'height': 1536, 'sensor': 'rgb', 'fps': 30, 'total_frames': 580, 'dataset': 1}
{'name': 'INCAR/20210422/Session1/C10_P7_P8_1/gt/INCAR_20210422_Session1_C10_P7_P8_1_rgb.mp4', 'id': 1, 'width': 2048, 'height': 1536, 'sensor': 'rgb', 'fps': 30, 'total_frames': 580, 'dataset': 1}
{'name': 'INCAR/20210422/Session1/C10_P7_P8_1/gt/INCAR_20210422_Session1_C10_P7_P8_1_rgb.mp4', 'id': 1, 'width': 2048, 'height': 1536, 'sensor': 'rgb', 'fps': 30, 'total_frames': 580, 'dataset': 1}


# OTHER FUNCTIONS

In [61]:
def replace_from_annotation(outdir_img, json_file, data, images, videos, categories, videos_ann, level):
    img_l=[]
    for x in tqdm(data['annotations'], desc='Annotations %s' % json_file):  
        # extract label and category
        catid = '%g' % x['category_id']
        label = catid
        category = categories[label]['name']
        # extract image info from x['image_id']
        image_id='%g' % x['image_id']
        if image_id in img_l: continue # continue to next loop if repeated image_id 
        img_l.append(image_id)
        img = images[image_id]
        h, w, imgf = img['height'], img['width'], img['file_name']
        _, img_ext = os.path.splitext(imgf)
        img_fn = Path(imgf).stem
        # extract video info from img['video_id']
        video_id = '%g' % img["video_id"]
        video = videos[video_id]
        videof= video["name"]
        video_fn = Path(videof).stem
        video_new_fn = video_fn #"video_"+video_id
        video_ann= videos_ann[video_id+'_'+catid]
        video_start_frame=video_ann["frame_start"]-1 # zero if starts in  1
        # src, dst
        img_old_fn = "img_"+img_fn.zfill(5) #"img_"+image_id
        img_frame=str(int(img_fn)-int(video_start_frame))
        img_new_fn = "img_"+img_frame.zfill(5) #img_framenumber (imgid with zeros 00001: image_frame.zfill(5) ) Problem is I don't no the maximum of images
        dirpath=os.path.join(outdir_img, video_new_fn)
        if level==2: dirpath = os.path.join(outdir_img, category, video_new_fn)
        src = os.path.join(dirpath, img_old_fn + img_ext)
        dst = os.path.join(dirpath, img_new_fn + img_ext)
        if src==dst: continue
        try:
            os.rename(src,dst)
        except:
            print("\n>> rename failed: {}".format(src))
            continue
    return
json_file='/home/administrator/server_data_ext/Recordings/EASYRIDE/P19/INCAR/mola.json'
outdir_img='/home/administrator/Z/Algorithms/mmaction2/data/INCAR/rawframes/'
level=2
with open(json_file) as f:
    data = json.load(f)
# create image dict {id: image}
images = {'%g' % x['id']: x for x in data['images']}
# create video dict {id: video}
videos = {'%g' % x['id']: x for x in data['videos']}
# create category dict {id: category}
categories = {'%g' % x['id']: x for x in data['categories']}
# create video ann dict {video_id_category_id: video_annotations}
videos_ann = {'%g' % x['video_id']+'_'+'%g' % x['category_id']: x for x in data['video_annotations']}


replace_from_annotation(outdir_img, json_file, data, images, videos, categories, videos_ann, level)

Annotations /home/administrator/server_data_ext/Recordings/EASYRIDE/P19/INCAR/mola.json:  65%|██████▍   | 288171/443433 [10:32<12:09, 212.80it/s]


>> rename failed: /home/administrator/Z/Algorithms/mmaction2/data/INCAR/rawframes/VIOLENT/INCAR_20210510_Session_1_C2_P1_P2_1_rgb/img_00464.png

>> rename failed: /home/administrator/Z/Algorithms/mmaction2/data/INCAR/rawframes/VIOLENT/INCAR_20210510_Session_1_C2_P1_P2_1_rgb/img_00465.png

>> rename failed: /home/administrator/Z/Algorithms/mmaction2/data/INCAR/rawframes/VIOLENT/INCAR_20210510_Session_1_C2_P1_P2_1_rgb/img_00466.png

>> rename failed: /home/administrator/Z/Algorithms/mmaction2/data/INCAR/rawframes/VIOLENT/INCAR_20210510_Session_1_C2_P1_P2_1_rgb/img_00467.png

>> rename failed: /home/administrator/Z/Algorithms/mmaction2/data/INCAR/rawframes/VIOLENT/INCAR_20210510_Session_1_C2_P1_P2_1_rgb/img_00468.png

>> rename failed: /home/administrator/Z/Algorithms/mmaction2/data/INCAR/rawframes/VIOLENT/INCAR_20210510_Session_1_C2_P1_P2_1_rgb/img_00469.png

>> rename failed: /home/administrator/Z/Algorithms/mmaction2/data/INCAR/rawframes/VIOLENT/INCAR_20210510_Session_1_C2_P1_P2_1_rgb

Annotations /home/administrator/server_data_ext/Recordings/EASYRIDE/P19/INCAR/mola.json:  65%|██████▍   | 288217/443433 [10:32<11:54, 217.28it/s]


>> rename failed: /home/administrator/Z/Algorithms/mmaction2/data/INCAR/rawframes/VIOLENT/INCAR_20210510_Session_1_C2_P1_P2_1_rgb/img_00510.png

>> rename failed: /home/administrator/Z/Algorithms/mmaction2/data/INCAR/rawframes/VIOLENT/INCAR_20210510_Session_1_C2_P1_P2_1_rgb/img_00511.png

>> rename failed: /home/administrator/Z/Algorithms/mmaction2/data/INCAR/rawframes/VIOLENT/INCAR_20210510_Session_1_C2_P1_P2_1_rgb/img_00512.png

>> rename failed: /home/administrator/Z/Algorithms/mmaction2/data/INCAR/rawframes/VIOLENT/INCAR_20210510_Session_1_C2_P1_P2_1_rgb/img_00513.png

>> rename failed: /home/administrator/Z/Algorithms/mmaction2/data/INCAR/rawframes/VIOLENT/INCAR_20210510_Session_1_C2_P1_P2_1_rgb/img_00514.png

>> rename failed: /home/administrator/Z/Algorithms/mmaction2/data/INCAR/rawframes/VIOLENT/INCAR_20210510_Session_1_C2_P1_P2_1_rgb/img_00515.png

>> rename failed: /home/administrator/Z/Algorithms/mmaction2/data/INCAR/rawframes/VIOLENT/INCAR_20210510_Session_1_C2_P1_P2_1_rgb

Annotations /home/administrator/server_data_ext/Recordings/EASYRIDE/P19/INCAR/mola.json:  65%|██████▌   | 288263/443433 [10:32<11:38, 222.24it/s]

...

Annotations /home/administrator/server_data_ext/Recordings/EASYRIDE/P19/INCAR/mola.json:  72%|███████▏  | 319714/443433 [13:02<10:14, 201.48it/s]


>> rename failed: /home/administrator/Z/Algorithms/mmaction2/data/INCAR/rawframes/VIOLENT/INCAR_20210510_Session_2_C3_P4_P1_2_rgb/img_00527.png

>> rename failed: /home/administrator/Z/Algorithms/mmaction2/data/INCAR/rawframes/VIOLENT/INCAR_20210510_Session_2_C3_P4_P1_2_rgb/img_00528.png

>> rename failed: /home/administrator/Z/Algorithms/mmaction2/data/INCAR/rawframes/VIOLENT/INCAR_20210510_Session_2_C3_P4_P1_2_rgb/img_00529.png

>> rename failed: /home/administrator/Z/Algorithms/mmaction2/data/INCAR/rawframes/VIOLENT/INCAR_20210510_Session_2_C3_P4_P1_2_rgb/img_00530.png

>> rename failed: /home/administrator/Z/Algorithms/mmaction2/data/INCAR/rawframes/VIOLENT/INCAR_20210510_Session_2_C3_P4_P1_2_rgb/img_00531.png

>> rename failed: /home/administrator/Z/Algorithms/mmaction2/data/INCAR/rawframes/VIOLENT/INCAR_20210510_Session_2_C3_P4_P1_2_rgb/img_00532.png

>> rename failed: /home/administrator/Z/Algorithms/mmaction2/data/INCAR/rawframes/VIOLENT/INCAR_20210510_Session_2_C3_P4_P1_2_rgb

Annotations /home/administrator/server_data_ext/Recordings/EASYRIDE/P19/INCAR/mola.json:  72%|███████▏  | 319757/443433 [13:02<10:03, 204.83it/s]


>> rename failed: /home/administrator/Z/Algorithms/mmaction2/data/INCAR/rawframes/VIOLENT/INCAR_20210510_Session_2_C3_P4_P1_2_rgb/img_00569.png

>> rename failed: /home/administrator/Z/Algorithms/mmaction2/data/INCAR/rawframes/VIOLENT/INCAR_20210510_Session_2_C3_P4_P1_2_rgb/img_00570.png

>> rename failed: /home/administrator/Z/Algorithms/mmaction2/data/INCAR/rawframes/VIOLENT/INCAR_20210510_Session_2_C3_P4_P1_2_rgb/img_00571.png

>> rename failed: /home/administrator/Z/Algorithms/mmaction2/data/INCAR/rawframes/VIOLENT/INCAR_20210510_Session_2_C3_P4_P1_2_rgb/img_00572.png

>> rename failed: /home/administrator/Z/Algorithms/mmaction2/data/INCAR/rawframes/VIOLENT/INCAR_20210510_Session_2_C3_P4_P1_2_rgb/img_00573.png

>> rename failed: /home/administrator/Z/Algorithms/mmaction2/data/INCAR/rawframes/VIOLENT/INCAR_20210510_Session_2_C3_P4_P1_2_rgb/img_00574.png

>> rename failed: /home/administrator/Z/Algorithms/mmaction2/data/INCAR/rawframes/VIOLENT/INCAR_20210510_Session_2_C3_P4_P1_2_rgb

Annotations /home/administrator/server_data_ext/Recordings/EASYRIDE/P19/INCAR/mola.json: 100%|██████████| 443433/443433 [25:47<00:00, 286.60it/s]


In [60]:
%debug

> <ipython-input-59-15aa2d463633>(32)replace_from_annotation()
     30         src = os.path.join(dirpath, img_old_fn + img_ext)
     31         dst = os.path.join(dirpath, img_new_fn + img_ext)
---> 32         if category=="VIOLENT": raise
     33         if src==dst: continue
     34         try:



ipdb>  video_ann


{'id': 1, 'category_id': 1, 'video_id': 1, 'time_start': 442, 'time_end': 580, 'label_frames': 139, 'dataset': 1}


ipdb>  img_new_fn


'img_00001'


ipdb>  quit


# FAILED CODE IDEAS

In [13]:
#WARNING TO hard to implement when you want to write images
#TO ADD IMGID you need to search for imgid on video
def write_from_video_annotation(data, images, videos, categories, copy_images, copy_videos, outdir_img, outdir_video):
    # WRITE FILES (COPY & GENERATE FILELIST)
    # video lists
    video_l = []
    saved_video_l= []
    videolist = []
    # write files 
    method="for" #TODO: parfor method
    start=time.time()
    if method=="for":
        #WRITE IMAGES
        for x in tqdm(data['video_annotations'], desc='Video Annotations %s' % json_file):  
            # extract video info from img['video_id']
            video_id = '%g' % x["video_id"]
            video = videos[video_id]
            videof= video["name"]
            video_sensor=video["sensor"]
            video_fn = Path(videof).stem
            video_new_fn = "video_"+video_id
            # extract label and category
            catid = '%g' % x['category_id']
            label = catid
            category = categories[catid]
            # extract total label frames
            total_frames = '%g' % x['label_frames']
            # extract category
            category = categories[label]['name']
            frame_start = x['frame_start'] #first frame
            frame_end = x['frame_end'] #end frame
            # extract image info from x['image_id']
            imgdir=os.path.dirname(videof)+'/'+video_sensor+'/'
            img_l= sorted( filter( os.path.isfile, glob.glob(imgdir + '*') ) )
            # write images - 1st because if copy_images fails the rest should not be done
            # image lists
            img_l = []
            saved_img_l= []
            imglist = []
            img_counter = 0 # image counter
            for imgd in img_l:
                src = os.path.join(imgd)
                dst = os.path.join(outdir_img, video_new_fn, img_new_fn + img_ext) #TODO: how to add imgid you need to search 
                if level==2: dst = os.path.join(outdir_img, category, video_new_fn, img_new_fn + img_ext)
                ext=extract_file(src,dst,copy=copy_images)
                if not ext: continue #if image missing from dataset when extracting images dont write nothing more
                # rawframe annotation file list: json to txt [ frame_directory total_frames label  ]
                imgline = f'{video_new_fn} {total_frames} {label}\n' # f'{video_new_fn}/{img_new_fn} {total_frames} {label}\n'
                if level==2: imgline = f'{category}/{video_new_fn} {total_frames} {label}\n' # f'{category}/{video_new_fn}/{img_new_fn} {total_frames} {label}\n'
                imglist.append(imgline)
                img_counter += 1
        #remove duplicate paths
        imglist=list(dict.fromkeys(imglist)) 
    stop = time.time()
    elapsed=stop-start
    print("time elapsed:", elapsed)
    return imglist, videolist, saved_img_l, saved_video_l